In [1]:
import functools

import numpy as np
import tensorflow as tf

In [11]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file('eval.csv', TEST_DATA_URL)


In [12]:
np.set_printoptions(precision = 3 , suppress= True)

In [16]:
LABAL_COLUMN = "survived"
LABELS = [0,1]

In [25]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
    file_path,
        batch_size=5, #배치사이즈를 작게 하면 보기 편하다
        label_name=LABAL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True,
        **kwargs)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [28]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key,value in batch.items():
            print(f"{key:20s}: {value.numpy()}")

In [29]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
age                 : [36. 18. 28. 23. 28.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [13.   73.5   7.75 13.    8.05]
class               : [b'Second' b'Second' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Queenstown' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'y']


In [32]:
CSV_COLUMNS = ['survived', 'sex','age', 'n_siblings_spouses', 'parch','fare','class','deck','embark_town','alone']

temp_dataset = get_dataset(train_file_path,column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'male' b'male' b'female']
age                 : [26. 28. 28. 28. 45.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 1]
fare                : [ 7.775 30.5   56.496  0.    14.454]
class               : [b'Third' b'First' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'C' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'y' b'y' b'y' b'y' b'n']


In [35]:
SELECT_COLUMNS = ['survived', 'age','n_siblings_spouses', 'class','deck','alone']

temp_dataset = get_dataset(train_file_path, select_columns = SELECT_COLUMNS)

show_batch(temp_dataset)



age                 : [28. 28. 28. 28. 28.]
n_siblings_spouses  : [1 0 0 0 0]
class               : [b'Third' b'Third' b'First' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [37]:
SELECT_COLUMNS = ['survived', 'age','n_siblings_spouses', 'parch','fare']

DEFAULTS = [0,0.0,0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, select_columns = SELECT_COLUMNS,column_defaults=DEFAULTS)

show_batch(temp_dataset)

age                 : [ 2. 36. 54. 49. 29.]
n_siblings_spouses  : [4. 0. 1. 0. 0.]
parch               : [1. 0. 0. 0. 0.]
fare                : [29.125 12.875 59.4    0.     7.775]


In [39]:
example_batch, labels_batch = next(iter(temp_dataset))

In [41]:
def pack(features, label):
    return tf.stack(list(features.values()), axis = -1), label

In [43]:
packed_dataset = temp_dataset.map(pack)
for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())
    

[[28.     0.     0.     7.229]
 [32.     0.     0.     8.05 ]
 [36.     0.     0.    12.875]
 [28.     0.     0.     7.896]
 [28.     1.     0.    15.5  ]]

[0 1 0 0 0]


In [45]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'female' b'male' b'male']
age                 : [19. 49. 24. 30.  4.]
n_siblings_spouses  : [0 1 0 0 1]
parch               : [0 0 0 0 1]
fare                : [10.171 76.729 69.3   13.    11.133]
class               : [b'Third' b'First' b'First' b'Second' b'Third']
deck                : [b'unknown' b'D' b'B' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'n']


In [46]:
example_batch, labels_batch = next(iter(temp_dataset))

In [48]:
example_batch

OrderedDict([('age',
              <tf.Tensor: id=576, shape=(5,), dtype=float32, numpy=array([48., 51., 28.,  4., 10.], dtype=float32)>),
             ('n_siblings_spouses',
              <tf.Tensor: id=578, shape=(5,), dtype=float32, numpy=array([0., 0., 0., 1., 3.], dtype=float32)>),
             ('parch',
              <tf.Tensor: id=579, shape=(5,), dtype=float32, numpy=array([0., 0., 0., 1., 2.], dtype=float32)>),
             ('fare',
              <tf.Tensor: id=577, shape=(5,), dtype=float32, numpy=array([26.55 ,  7.054,  8.113, 23.   , 27.9  ], dtype=float32)>)])

In [53]:
class PackNumericFeatures(object):  #좀더전처리를 쉽게 해주기 위ㅐ서 class로 선언을 해주자
    def __init__(self, names):
        self.names= names
        
    def  __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        
        return features, labels
    

In [54]:
NUMERIC_FEATURES = ['age', "n_siblings_spouses", 'parch','fare']
packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))


In [55]:
show_batch(packed_train_data)

sex                 : [b'female' b'female' b'male' b'male' b'female']
class               : [b'First' b'Third' b'Second' b'First' b'Second']
deck                : [b'A' b'unknown' b'unknown' b'A' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'y']
numeric             : [[48.     1.     0.    39.6  ]
 [63.     0.     0.     9.587]
 [30.     0.     0.    10.5  ]
 [40.     0.     0.    31.   ]
 [28.     0.     0.    13.   ]]


In [57]:
example_batch, labels_batch = next(iter(packed_train_data))

### Data Normalization

In [58]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()

In [59]:
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [63]:
MEAN = np.array(desc.T['mean'])
MEAN

array([29.631,  0.545,  0.38 , 34.385])

In [65]:
STD = np.array(desc.T['std'])
STD

array([12.512,  1.151,  0.793, 54.598])

In [66]:
def normalize_numeric_data(data, mean, std):
    return (data-mean)/std

In [67]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000002A6F3C8CB70>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [68]:
example_batch['numeric']

<tf.Tensor: id=675, shape=(5, 4), dtype=float32, numpy=
array([[ 58.   ,   0.   ,   0.   , 146.521],
       [ 37.   ,   1.   ,   0.   ,  53.1  ],
       [ 24.   ,   0.   ,   0.   ,  13.   ],
       [ 65.   ,   0.   ,   1.   ,  61.979],
       [ 22.   ,   0.   ,   0.   ,  10.517]], dtype=float32)>

In [69]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 2.267, -0.474, -0.479,  2.054],
       [ 0.589,  0.395, -0.479,  0.343],
       [-0.45 , -0.474, -0.479, -0.392],
       [ 2.827, -0.474,  0.782,  0.505],
       [-0.61 , -0.474, -0.479, -0.437]], dtype=float32)

In [70]:
CATEGORIES = { 'sex': ['male','female'],
             'class':['First','Second','Third'],
             'deck':['A','B','C','D','E','F','G','H',"I",'J'],
             'embark_town' : ['Cherbourg','SOuthhampton','Queenstown'],
             'alone':['y','n']
             }

In [71]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [72]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'SOuthhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [74]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]


In [75]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     1.     0.     0.     0.     1.     0.     0.     0.
  0.     0.     0.     0.     0.     1.     0.     0.     2.267 -0.474
 -0.479  2.054  0.     1.   ]


## build the model

In [77]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation= "relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1),
])

In [79]:
model.compile(
loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
optimizer = 'adam',
metrics=['accuracy'])

In [80]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [81]:
model.fit(train_data, epochs = 20)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
126/126 [==============================] - 2s 17ms/step - loss: 0.4992 - accuracy: 0.7528
Epoch 2/20
126/126 [==============================] - 1s 4ms/step - loss: 0.4244 - accuracy: 0.8102
Epoch 3/20
126/126 [==============================] - 1s 4ms/step - loss: 0.4029 - accuracy: 0.8293
Epoch 4/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3985 - accuracy: 0.8341
Epoch 5/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3807 - accuracy: 0.8278
Epoch 6/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3774 - accuracy: 0.8309
Epoch 7/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3639 - accuracy: 0.8373
Epoch 8/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3666 - accuracy: 0.8421
Epoch 9/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3564 - accuracy: 0.8453
Epoch

In [83]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f'\n\n Test loss {test_loss} , Test Accuravy{test_accuracy}')

53/53 [==============================] - 0s 6ms/step - loss: 0.4543 - accuracy: 0.8447


 Test loss 0.4543059452102994 , Test Accuravy0.8446969985961914


In [89]:
predictions =model.predict(test_data)

for predictions, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    predictions = tf.sigmoid(predictions).numpy()
    print(f"Predicted survival: { predictions[0]:.2%}", '실제 값 : ', {"Survived " if bool(survived) else "DIDE"})

Predicted survival: 22.22% 실제 값 :  {'Survived '}
Predicted survival: 50.23% 실제 값 :  {'DIDE'}
Predicted survival: 54.19% 실제 값 :  {'DIDE'}
Predicted survival: 15.22% 실제 값 :  {'DIDE'}
Predicted survival: 98.14% 실제 값 :  {'Survived '}
